In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #type: ignore
from tensorflow.keras.applications import MobileNetV2 #type: ignore
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense #type: ignore
from tensorflow.keras.models import Model #type: ignore
from tensorflow.keras.optimizers import Adam #type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint #type: ignore

In [ ]:
# --- Settings & Paths ---
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25
DATA_DIR = "E:\\CAI 2.0\\Deep Learning\\Projects\\recycle_vision\\data\\garbage-dataset"   # Path to your dataset directory, in 'class folders' format
MODEL_PATH = "model/waste_model.keras"
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)

In [20]:
# --- DATA AUGMENTATION ---
datagen = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    brightness_range=[0.7, 1.3],
    shear_range=0.15,
    zoom_range=0.2
)

In [ ]:
# --- Datasets ---
train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=RANDOM_SEED
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=True,
    seed=RANDOM_SEED
)

Found 15813 images belonging to 10 classes.
Found 3949 images belonging to 10 classes.


In [22]:
# --- Model Architecture ---
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights="imagenet")
base_model.trainable = False  # Freeze base for initial transfer learning

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
output = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["accuracy"])

print('\n=== MODEL SUMMARY ===\n')
model.summary()


=== MODEL SUMMARY ===



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,423,242 (9.24 MB)

 Trainable params: 165,258 (645.54 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [23]:
# --- Callbacks ---
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1),
    ModelCheckpoint(MODEL_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [24]:
# --- Training ---
print("\n--- Training model (transfer learning, base frozen) ---\n")
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)


--- Training model (transfer learning, base frozen) ---



c:\Users\mysterious\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8052 - loss: 0.6219
Epoch 1: val_accuracy improved from -inf to 0.88098, saving model to model/waste_model.keras
495/495 ━━━━━━━━━━━━━━━━━━━━ 1125s 2s/step - accuracy: 0.8054 - loss: 0.6215 - val_accuracy: 0.8810 - val_loss: 0.3514 - learning_rate: 0.0010
Epoch 2/25
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9033 - loss: 0.2912
Epoch 2: val_accuracy improved from 0.88098 to 0.88681, saving model to model/waste_model.keras
495/495 ━━━━━━━━━━━━━━━━━━━━ 989s 2s/step - accuracy: 0.9033 - loss: 0.2912 - val_accuracy: 0.8868 - val_loss: 0.3316 - learning_rate: 0.0010
Epoch 3/25
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9218 - loss: 0.2319
Epoch 3: val_accuracy improved from 0.88681 to 0.88807, saving model to model/waste_model.keras
495/495 ━━━━━━━━━━━━━━━━━━━━ 1030s 2s/step - accuracy: 0.9218 - loss: 0.2319 - val_accuracy: 0.8881 - val_loss: 0.3452 - learning_rate: 0.0010
Epoch 4/25
495/495 ━━━━━━━━━━━━━

In [25]:
# --- Optional: Fine-tune some base_model layers after initial training ---
print("\n--- Fine-tuning base model (optional, improves accuracy) ---\n")
base_model.trainable = True


--- Fine-tuning base model (optional, improves accuracy) ---



In [ ]:
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8137 - loss: 0.6041
Epoch 1: val_accuracy did not improve from 0.91719
495/495 ━━━━━━━━━━━━━━━━━━━━ 3048s 6s/step - accuracy: 0.8137 - loss: 0.6038 - val_accuracy: 0.8949 - val_loss: 0.3585 - learning_rate: 1.0000e-05
Epoch 2/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.8962 - loss: 0.3033
Epoch 2: val_accuracy did not improve from 0.91719
495/495 ━━━━━━━━━━━━━━━━━━━━ 3246s 7s/step - accuracy: 0.8962 - loss: 0.3033 - val_accuracy: 0.9053 - val_loss: 0.3217 - learning_rate: 1.0000e-05
Epoch 3/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9242 - loss: 0.2273
Epoch 3: val_accuracy did not improve from 0.91719
495/495 ━━━━━━━━━━━━━━━━━━━━ 3114s 6s/step - accuracy: 0.9242 - loss: 0.2273 - val_accuracy: 0.9109 - val_loss: 0.2933 - learning_rate: 1.0000e-05
Epoch 4/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.9360 - loss: 0.1965
Epoch 4: val_accuracy did not improve from 0.91719
495/495 ━━━━━━━

In [27]:
# --- Save Final Model ---
model.save(MODEL_PATH, save_format="keras")
print(f"\n✅ Model successfully saved to: {MODEL_PATH}")


✅ Model successfully saved to: model/waste_model.keras


In [28]:
# --- Export Class Indices (handy for deployment) ---
label_map = train_gen.class_indices
import json
with open(os.path.join(os.path.dirname(MODEL_PATH), "label_map.json"), "w") as f:
    json.dump(label_map, f)
print("\nClass indices saved for deployment.\n")



Class indices saved for deployment.

